In [ ]:
# Notebook 2: Data Cleaning

import os
import pandas as pd


# --- 2. Load raw files and clean ---
# Load the layout for column names
orig_layout = pd.read_excel('../data/file_layout.xlsx', sheet_name='Origination Data File', skiprows=1)
perf_layout = pd.read_excel('../data/file_layout.xlsx', sheet_name='Monthly Performance Data File', skiprows=1)
orig_cols = orig_layout['ATTRIBUTE NAME'].tolist()
perf_cols = perf_layout['ATTRIBUTE NAME'].tolist()
print(f"orig cols={orig_cols}")
print(f"perf cols={perf_cols}")
print(f"orig cols={len(orig_cols)}")
print(f"perf cols={len(perf_cols)}")



orig cols=['Credit Score', 'First Payment Date', 'First Time Homebuyer Flag', 'Maturity Date', 'Metropolitan Statistical Area (MSA) Or Metropolitan Division', 'Mortgage Insurance Percentage (MI %)', 'Number of Units', 'Occupancy Status', 'Original Combined Loan-to-Value (CLTV)', 'Original Debt-to-Income (DTI) Ratio', 'Original UPB', 'Original Loan-to-Value (LTV)', 'Original Interest Rate', 'Channel', 'Prepayment Penalty Mortgage (PPM) Flag', 'Amortization Type (Formerly Product Type)', 'Property State', 'Property Type', 'Postal Code', 'Loan Sequence Number', 'Loan Purpose', 'Original Loan Term', 'Number of Borrowers', 'Seller Name', 'Servicer Name', 'Super Conforming Flag', 'Pre-HARP Loan Sequence Number', 'Program Indicator', 'HARP Indicator', 'Property Valuation Method', 'Interest Only (I/O) Indicator', 'Mortgage Insurance Cancellation Indicator']
perf cols=['Loan Sequence Number', 'Monthly Reporting Period', 'Current Actual UPB', 'Current Loan Delinquency Status', 'Loan Age', 'Remai

In [8]:
for year in range(1999, 2025):
    # Paths to raw txt files
    orig_path = f"../data/raw/origination/sample_orig_{year}.txt"
    perf_path = f"../data/raw/performance/sample_perf_{year}.txt"
    if not os.path.exists(orig_path) or not os.path.exists(perf_path):
        continue  # Skip if files not present

    # Read as strings to apply cleaning
    orig_df = pd.read_csv(orig_path, sep='|', names=orig_cols, dtype=str, low_memory=False)
    perf_df = pd.read_csv(perf_path, sep='|', names=perf_cols, dtype=str, low_memory=False)

    # Trim whitespace for all string columns
    orig_df = orig_df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
    perf_df = perf_df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
    # # print(perf_df.head())
    # print(perf_df.columns)

    # Convert numeric fields and handle special codes
    # Example: Credit Score (9999 -> NaN) and DTI (999 -> NaN)
    orig_df['Credit Score'] = pd.to_numeric(orig_df['Credit Score'], errors='coerce')
    orig_df.loc[orig_df['Credit Score'] == 9999, 'Credit Score'] = pd.NA  # 9999 = Not Available:contentReference[oaicite:4]{index=4}

    orig_df['Original Combined Loan-to-Value (CLTV)'] = pd.to_numeric(orig_df['Original Combined Loan-to-Value (CLTV)'], errors='coerce')
    orig_df.loc[orig_df['Original Combined Loan-to-Value (CLTV)'] == 999, 'Original Combined Loan-to-Value (CLTV)'] = pd.NA  # 999 = NA:contentReference[oaicite:5]{index=5}

    orig_df['Original Debt-to-Income (DTI) Ratio'] = pd.to_numeric(orig_df['Original Debt-to-Income (DTI) Ratio'], errors='coerce')
    orig_df.loc[orig_df['Original Debt-to-Income (DTI) Ratio'] == 999, 'Original Debt-to-Income (DTI) Ratio'] = pd.NA  # 999 = NA:contentReference[oaicite:6]{index=6}

    # Parse date fields (YYYYMM)
    for date_col in ['First Payment Date', 'Maturity Date']:
        orig_df[date_col] = pd.to_datetime(orig_df[date_col], format='%Y%m', errors='coerce')

    # Performance data cleaning
    # Convert numeric
    perf_df['Loan Age'] = pd.to_numeric(perf_df['Loan Age'], errors='coerce')
    perf_df['Current Actual UPB'] = pd.to_numeric(perf_df['Current Actual UPB'], errors='coerce')
    perf_df['Current Loan Delinquency Status'] = pd.to_numeric(perf_df['Current Loan Delinquency Status'], errors='coerce')
    perf_df['Zero Balance Code'] = pd.to_numeric(perf_df['Zero Balance Code'], errors='coerce')
    perf_df['Zero Balance Effective Date'] = pd.to_numeric(perf_df['Zero Balance Effective Date'], errors='coerce')
    perf_df['Zero Balance Effective Date'] = pd.to_datetime(perf_df['Zero Balance Effective Date'], format='%Y%m', errors='coerce')
    perf_df['Monthly Reporting Period'] = pd.to_datetime(perf_df['Monthly Reporting Period'], format='%Y%m', errors='coerce')

    # Handle flags in orig: '9' -> NaN
    orig_df['First Time Homebuyer Flag'] = orig_df['First Time Homebuyer Flag'].replace({'9': pd.NA})
    orig_df['Occupancy Status'] = orig_df['Occupancy Status'].replace({'9': pd.NA})
    orig_df['Loan Purpose'] = orig_df['Loan Purpose'].replace({'Z': pd.NA})  # if 'Z' indicates NA in loan purpose

    # Save cleaned data to interim (Parquet format)
    orig_df.to_parquet(f"../data/interim/origination_{year}.parquet", index=False)
    perf_df.to_parquet(f"../data/interim/performance_{year}.parquet", index=False)
    print(f"Year {year} cleaned and saved.")

Year 1999 cleaned and saved.
Year 2000 cleaned and saved.
Year 2001 cleaned and saved.
Year 2002 cleaned and saved.
Year 2003 cleaned and saved.
Year 2004 cleaned and saved.
Year 2005 cleaned and saved.
Year 2006 cleaned and saved.
Year 2007 cleaned and saved.
Year 2008 cleaned and saved.
Year 2009 cleaned and saved.
Year 2010 cleaned and saved.
Year 2011 cleaned and saved.
Year 2012 cleaned and saved.
Year 2013 cleaned and saved.
Year 2014 cleaned and saved.
Year 2015 cleaned and saved.
Year 2016 cleaned and saved.
Year 2017 cleaned and saved.
Year 2018 cleaned and saved.
Year 2019 cleaned and saved.
Year 2020 cleaned and saved.
Year 2021 cleaned and saved.
Year 2022 cleaned and saved.
Year 2023 cleaned and saved.
Year 2024 cleaned and saved.
